In [1]:
from pyspark.sql.functions import dense_rank,col,avg,to_date,round as round_,current_date,to_date as to_date_,months_between,udf
from pyspark.sql.window import Window

from pyspark.sql import SparkSession
from delta import *
from pyspark.sql.types import ShortType

# Import dataframe into MySQL
import sqlalchemy


In [2]:

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.jars", "/usr/local/lib/postgresql-42.2.5.jar")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()



udf_flag_own_car = udf(lambda x: 0 if x == 'N' else 1,ShortType() )

In [13]:
df_features = spark.read.csv('dataset_credit_risk.csv',inferSchema =True, header=True).na.drop()\
.sort(col('id'),col('loan_date'))\
.withColumn("loan_date",to_date(col("loan_date"),"yyyy-MM-dd"))\
.withColumn("nb_previous_loans", dense_rank().over(Window.partitionBy("id").orderBy(col("loan_date")))-1)\
.sort(col('id'),col('loan_date'))\
.withColumn('avg_amount_loans_previous', avg(col('loan_amount'))
            .over(Window.partitionBy(col('id')).orderBy(col("loan_date")).rowsBetween(Window.unboundedPreceding,-1)))\
.withColumn("age",round_(months_between(current_date(),to_date_(col("birthday"), "yyyy-MM-dd"), True)/12).cast('int'))\
.withColumn("years_on_the_job",round_(months_between(current_date(),to_date(col("job_start_date"), "yyyy-MM-dd"), True)/12).cast('int'))\
.withColumn("flag_own_car",udf_flag_own_car(col('flag_own_car')))\


#------------------------------------------------
# Features To MySQL DB 
#-------------------------------------------------
df_features.select('id', 'age', 'years_on_the_job', 'nb_previous_loans', 'avg_amount_loans_previous', 'flag_own_car')\
.write.format("jdbc").mode("append")\
.option("url", "jdbc:postgresql://172.17.0.3:5432/kueski") \
.option("driver", "org.postgresql.Driver").option("dbtable", "features") \
.option("user", "postgres").option("password", "postgresmaster").save()\

#------------------------------------------------
# Features To CSV for model training
#-------------------------------------------------
df_features.select('id', 'age', 'years_on_the_job', 'nb_previous_loans', 'avg_amount_loans_previous', 'flag_own_car', 'status')\
.repartition(1).write.format("csv").option("header", "true").save("train.csv")

NameError: name 'spark' is not defined